In [5]:
#importing Libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import seaborn as sns
from sklearn.metrics import r2_score
import pprint
from rdkit import Chem
from rdkit.Chem import Descriptors

In [10]:
#Reading the data & dropping uneeded columns

data = pd.read_csv("/Users/iainquinn/Crystalisation Internship/Data/DataRaw.csv")
data = data[[ "smiles", "Source", "Method", "Molecule", "solvent","induction_time" ,"Supersaturation"]].copy()

#Making two new columns

data['ln_t'] = np.log(data['induction_time'])
data['ln_(S-1)'] = np.log(data['Supersaturation']-1)

#Adding a new column called molecule-source

moleculesource_lst=[]
solutesolvent_lst=[]

for index, row in data.iterrows():
    moleculesource_lst.append(row['Molecule'] + ' ' + row['Source'])
data['molecule_source'] = moleculesource_lst

for index, row in data.iterrows():
    solutesolvent_lst.append(row['Molecule'] + ' (' + str(row['solvent']) + ')')
data['solute_solvent'] = solutesolvent_lst

#making two new datasets

data_anti = data.loc[data.Method == "Antisolvent"]
data_cool = data.loc[data.Method == "Cooling"]

#Showing the simplified data
#data.head()

In [11]:
#Making lists of each unique molecule in the datasheet

molecule_list = data.Molecule.unique()
molecule_list_anti = data_anti.Molecule.unique()
molecule_list_cool = data_cool.Molecule.unique()

#Making a list of each unique molecule-source object in the datasheet

molsource_list = np.unique(moleculesource_lst)
molsource_list_anti = data_anti.molecule_source.unique()
molsource_list_cool = data_cool.molecule_source.unique()
solutesolvent_lst=data.solute_solvent.unique()

df=pd.DataFrame(solutesolvent_lst)
df.to_csv('solutes.csv')

In [4]:
molecules_source=[]
score=[]
slope=[]
intercept=[]
median=[]
method=[]
smiles=[]

for entry in molsource_list_anti:
    data_set = data.loc[data.molecule_source == entry]
    Y = np.c_[data_set["ln_t"]]
    X = np.c_[data_set["ln_(S-1)"]]
    model = sklearn.linear_model.LinearRegression()
    model.fit(X, Y)
    prediction=model.predict(X)
    molecules_source.append(entry)
    score.append(r2_score(Y, prediction))
    slope.append(float(model.coef_))
    intercept.append(float(model.intercept_))
    median.append(data_set["Supersaturation"].median())
    method.append('antisolvent')
    smiles.append(str(data_set.iloc[0]['smiles']))
    
df_anti = pd.DataFrame(list(zip( molecules_source, slope, intercept, score, median, method, smiles)) , columns=['molecule_source','slope', 'intercept', 'r2','median', 'method', 'smiles'])

molecules_source=[]
score=[]
slope=[]
intercept=[]
median=[]
method=[]
smiles=[]

for entry in molsource_list_cool:
    data_set = data.loc[data.molecule_source == entry]
    Y = np.c_[data_set["ln_t"]]
    X = np.c_[data_set["ln_(S-1)"]]
    model = sklearn.linear_model.LinearRegression()
    model.fit(X, Y)
    prediction=model.predict(X)
    molecules_source.append(entry)
    score.append(r2_score(Y, prediction))
    slope.append(float(model.coef_))
    intercept.append(float(model.intercept_))
    median.append(data_set["Supersaturation"].median())
    method.append('cooling')
    smiles.append(str(data_set.iloc[0]['smiles']))
    
df_cool = pd.DataFrame(list(zip( molecules_source, slope, intercept, score, median, method, smiles)) , columns=['molecule_source','slope', 'intercept', 'r2','median', 'method', 'smiles'])

df = pd.concat([df_anti, df_cool], ignore_index=True)
df.head(50)


,molecule_source,slope,intercept,r2,median,method,smiles
0,Abecarnil Beckmann_1999,-2.679235,3.495162,0.901418,1.773416,antisolvent,CC(C)OC(=O)C1=NC=C2C(=C1COC)C3=C(N2)C=CC(=C3)O...
1,Benzoic acid Zhao_2019,-1.054949,2.597384,0.099874,1.346847,antisolvent,C1=CC=C(C=C1)C(=O)O
2,Cefodizime sodium Zhang_2013,-0.747076,5.079866,0.123162,2.247142,antisolvent,CC1=C(SC(=N1)SCC2=C(N3C(C(C3=O)NC(=O)C(=NOC)C4...
3,Cefuroxime Sodium Zhao_2012,-2.992770,3.383713,0.894575,1.457791,antisolvent,CON=C(C1=CC=CO1)C(=O)NC2C3N(C2=O)C(=C(CS3)COC(...
4,Dexamethasone Hao_2005,-1.412730,5.591404,0.256615,2.082191,antisolvent,CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CO)O...
5,Erythromycin ethylsuccinate Li_2016,-0.580725,6.293097,0.112606,1.488132,antisolvent,CCC1C(C(C(C(=O)C(CC(C(C(C(C(C(=O)O1)C)OC2CC(C(...
6,Glycine Forsyth_2015,-4.150214,3.092932,0.012060,1.400000,antisolvent,C(C(=O)O)N
7,Glycine Ramakers 2020,-2.103595,4.009274,0.731073,1.370000,antisolvent,C(C(=O)O)N
8,L-Histidine Liu_2018,-1.120890,5.368310,0.817177,3.112457,antisolvent,C1=C(NC=N1)CC(C(=O)O)N
9,Naproxen Poornachary_2016,-5.140402,4.559187,0.358861,1.989169,antisolvent,CC(C1=CC2=C(C=C1)C=C(C=C2)OC)C(=O)O


In [17]:
#Splitting the dataset into two based on how good the R2 value is

df_poor = df.loc[df.r2 < 0.5]
df_good = df.loc[df.r2 > 0.5]

#Making a list of the molecule-source in each dataset

poor_list = df_poor['molecule_source'].tolist()
good_list = df_good['molecule_source'].tolist()

df_good = df_good.reset_index(drop=True)
df_good=df_good.drop(columns=['smiles'])
df_good.head(40)

,molecule_source,slope,intercept,r2,median,method
0,Abecarnil Beckmann_1999,-2.679235,3.495162,0.901418,1.773416,antisolvent
1,Cefuroxime Sodium Zhao_2012,-2.992770,3.383713,0.894575,1.457791,antisolvent
2,Glycine Ramakers 2020,-2.103595,4.009274,0.731073,1.370000,antisolvent
3,L-Histidine Liu_2018,-1.120890,5.368310,0.817177,3.112457,antisolvent
4,Paracetamol O'Ciardha_2011,-2.393867,4.126005,0.567710,1.245241,antisolvent
5,Paracetamol Kodera_2019,-1.217616,9.020724,0.997630,3.081787,antisolvent
6,Potash Alum Mydlarz_1991,-1.763159,5.111083,0.906718,3.834832,antisolvent
7,Salicylic acid Wang_2005,-2.832536,2.698066,0.662246,1.354483,antisolvent
8,Sodium Chloride Takiyama_1998,-0.249183,1.447489,0.663801,1.735955,antisolvent
9,Cytidine 5-monophosphate disodium salt (5-CMPN...,-0.773099,4.805979,0.739086,1.585000,cooling


In [6]:
#Making a list of all smiles

smiles_list=df_good.smiles.tolist()
pprint.pprint(smiles_list)

['CC(C)OC(=O)C1=NC=C2C(=C1COC)C3=C(N2)C=CC(=C3)OCC4=CC=CC=C4',
 'CON=C(C1=CC=CO1)C(=O)NC2C3N(C2=O)C(=C(CS3)COC(=O)N)C(=O)[O-].[Na+]',
 'C(C(=O)O)N',
 'C1=C(NC=N1)CC(C(=O)O)N',
 'CC(=O)NC1=CC=C(C=C1)O',
 'CC(=O)NC1=CC=C(C=C1)O',
 '[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[Al+3].[K+]',
 'C1=CC=C(C(=C1)C(=O)O)O',
 '[Na+].[Cl-]',
 'C1=CN(C(=O)N=C1N)C2C(C(C(O2)COP(=O)(O)O)O)O',
 'C1=CC(=CC(=C1)O)C(=O)O',
 'CC(=O)NC1=CC=C(C=C1)O',
 'CC(=O)NC1=CC=C(C=C1)O',
 'CC(=O)NC1=CC=C(C=C1)O',
 'B(=O)[O-].O.O.[Na+]',
 '[O-][Mo](=O)(=O)[O-].[Na+].[Na+]',
 'C(C1C(C(C(C(O1)O)O)O)O)O',
 'S(=O)(=O)([O-])[O-].[K+].[K+]',
 'O.O.O.O.O.O.O.O.O.O.B([O-])([O-])[O-].B([O-])([O-])[O-].B([O-])([O-])[O-].B([O-])([O-])[O-].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+].[Na+]']


In [7]:
mol_descriptors = [desc[0] for desc in Descriptors.descList]
dict_list=[]

for smile in smiles_list:
    my_dict={}
    for desc in mol_descriptors:
        a = "b=Descriptors." + desc + "(Chem.MolFromSmiles(smile))"
        exec(a)
        my_dict[desc]=b
    my_dict['smiles']=smile
    dict_list.append(my_dict)
        
df_desc = pd.DataFrame(dict_list)
df_desc.head(20)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,smiles
0,12.603529,-0.453905,12.603529,0.230232,0.438589,404.466,380.274,404.173607,154,0,...,0,0,0,0,0,0,0,0,0,CC(C)OC(=O)C1=NC=C2C(=C1COC)C3=C(N2)C=CC(=C3)O...
1,12.542599,-1.603040,12.542599,0.000000,0.179901,446.373,431.253,446.050829,154,0,...,0,0,0,0,0,0,0,0,0,CON=C(C1=CC=CO1)C(=O)NC2C3N(C2=O)C(=C(CS3)COC(...
2,9.243056,-0.967593,9.243056,0.277778,0.421171,75.067,70.027,75.032028,30,0,...,0,0,0,0,0,0,0,0,0,C(C(=O)O)N
3,10.266220,-1.000370,10.266220,0.287037,0.541194,155.157,146.085,155.069477,60,0,...,0,0,0,0,0,0,0,0,0,C1=C(NC=N1)CC(C(=O)O)N
4,10.524469,-0.115102,10.524469,0.115102,0.595026,151.165,142.093,151.063329,58,0,...,0,0,0,0,0,0,0,0,0,CC(=O)NC1=CC=C(C=C1)O
5,10.524469,-0.115102,10.524469,0.115102,0.595026,151.165,142.093,151.063329,58,0,...,0,0,0,0,0,0,0,0,0,CC(=O)NC1=CC=C(C=C1)O
6,8.520833,-5.166667,8.520833,0.000000,0.235377,258.206,258.206,257.848704,64,0,...,0,0,0,0,0,0,0,0,0,[O-]S(=O)(=O)[O-].[O-]S(=O)(=O)[O-].[Al+3].[K+]
7,10.261759,-1.112870,10.261759,0.067130,0.610259,138.122,132.074,138.031694,52,0,...,0,0,0,0,0,0,0,0,0,C1=CC=C(C(=C1)C(=O)O)O
8,0.000000,0.000000,0.000000,0.000000,0.243392,58.443,58.443,57.958622,8,0,...,0,0,0,0,0,0,0,0,0,[Na+].[Cl-]
9,11.646408,-4.742363,11.646408,0.022530,0.373590,323.198,309.086,323.051851,118,0,...,0,0,0,0,0,0,0,0,0,C1=CN(C(=O)N=C1N)C2C(C(C(O2)COP(=O)(O)O)O)O


In [21]:
df=df_good.join(df_desc)

In [22]:
corr_matrix= df.corr()
corr=corr_matrix["slope"].sort_values(ascending=False)
corr[0:20]

slope               1.000000
intercept           0.623847
BCUT2D_MRHI         0.476903
fr_aniline          0.475998
SlogP_VSA10         0.426702
BCUT2D_MWHI         0.411075
median              0.410247
fr_halogen          0.379030
fr_NH1              0.347851
qed                 0.332088
BalabanJ            0.326669
NumHDonors          0.321227
r2                  0.308319
EState_VSA6         0.296750
VSA_EState4         0.295632
PEOE_VSA12          0.290061
FpDensityMorgan3    0.287865
fr_ArN              0.286059
fr_phos_acid        0.286059
fr_phos_ester       0.286059
Name: slope, dtype: float64

In [23]:
corr_matrix= df.corr()
corr=corr_matrix["slope"].sort_values(ascending=True)
corr[0:10]

BCUT2D_MRLOW    -0.300314
PEOE_VSA14      -0.273486
HallKierAlpha   -0.239884
SMR_VSA1        -0.239712
EState_VSA2     -0.229008
SlogP_VSA1      -0.218077
fr_Ar_COO       -0.216064
EState_VSA9     -0.206749
VSA_EState1     -0.196166
Kappa2          -0.187278
Name: slope, dtype: float64

In [24]:
corr_matrix= df.corr()
corr=corr_matrix["median"].sort_values(ascending=False)
corr[0:10]

median                 1.000000
MaxAbsPartialCharge    0.533690
PEOE_VSA8              0.530252
MaxPartialCharge       0.477536
fr_imidazole           0.447682
VSA_EState1            0.415509
slope                  0.410247
r2                     0.395398
intercept              0.390080
fr_Ar_NH               0.343779
Name: median, dtype: float64

In [25]:
corr_matrix= df.corr()
corr=corr_matrix["median"].sort_values(ascending=True)
corr[0:10]

VSA_EState9                -0.271978
VSA_EState3                -0.237038
MinPartialCharge           -0.186003
NHOHCount                  -0.176019
NumSaturatedHeterocycles   -0.160887
NumSaturatedRings          -0.160887
fr_Al_OH                   -0.158134
fr_Al_OH_noTert            -0.158134
SMR_VSA6                   -0.155114
Kappa2                     -0.152529
Name: median, dtype: float64